# Latency Lab Analysis

This notebook reads `results/index.csv` and shows basic comparisons.


In [ ]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "results_lib.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
sys.path.append(str(repo_root / "scripts"))

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from results_lib import load_index, load_samples

sns.set_theme(style="whitegrid")


## Run benchmarks from the notebook


In [ ]:
import sys
from pathlib import Path

from IPython.display import display

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "notebook_runner.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import importlib
import notebook_ui
importlib.reload(notebook_ui)

bench_path = repo_root / "build" / "bench"
results_dir = repo_root / "results"


In [ ]:
ui, output = notebook_ui.display_runner_ui(
    bench_path=bench_path,
    results_dir=results_dir,
)


In [ ]:
index = load_index(repo_root / "results" / "index.csv")

if hasattr(index, "columns") and "run_dir" in index.columns:
    def _abs_run_dir(value):
        if not isinstance(value, str) or not value:
            return value
        path = Path(value)
        if path.is_absolute():
            return value
        return str((repo_root / path).resolve())

    index["run_dir"] = index["run_dir"].apply(_abs_run_dir)

index.head()


In [ ]:
# Pick a case to compare pinned vs unpinned (adjust as needed)
case_name = index["case"].iloc[0] if len(index) else "noop"
subset = index[index["case"] == case_name]

display(subset[["case","tags","pin_cpu","p50","p95","p99","p999","mean"]])


In [ ]:
# Long-form view of quantiles for the chosen case
if len(subset):
    plot_df = subset.melt(
        id_vars=["tags","pin_cpu"],
        value_vars=["p50","p95","p99","p999"],
        var_name="quantile",
        value_name="ns",
    )
    plt.figure(figsize=(8,4))
    sns.barplot(data=plot_df, x="quantile", y="ns", hue="pin_cpu")
    plt.title(f"{case_name} quantiles by pin_cpu")
    plt.show()


In [ ]:
# ECDF of raw samples for the first two runs (if raw data is present)
def short_label(value):
    if not isinstance(value, str):
        return str(value)
    path = Path(value)
    parts = path.parts
    if "results" in parts:
        idx = parts.index("results")
        tail = parts[idx + 1 :]
        if len(tail) >= 3:
            return "/".join(tail[:3])
        if tail:
            return "/".join(tail)
    try:
        return str(path.relative_to(repo_root))
    except Exception:
        return path.name or str(value)

if len(index) >= 2:
    run_a = index.iloc[0]["run_dir"]
    run_b = index.iloc[1]["run_dir"]
    samples_a = load_samples(run_a)
    samples_b = load_samples(run_b)

    plt.figure(figsize=(12,8))
    sns.ecdfplot(samples_a, label=short_label(run_a))
    sns.ecdfplot(samples_b, label=short_label(run_b))
    plt.xlabel("ns")
    plt.ylabel("ECDF")
    plt.legend()
    plt.title("ECDF comparison")
    plt.show()
